In [22]:
import numpy as np
import dill

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from Rpfa import micro_rpfa

import logging
import datetime
import pickle

from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

DATASET = "CoralBleaching"
PARTITION = "Test"

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
training_folder = root_folder + PARTITION + "/"
training_pickled = settings.data_directory + DATASET + "/Thesis_Dataset/" + PARTITION.lower() + ".pl"

root_folder

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


'/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/'

In [23]:
config = get_config(training_folder)
# override this so we don't replace INFREQUENT words
config["min_df"] = 0
tagged_essays2 = load_process_essays(**config)
#config

226 files found
226 essays processed


In [24]:
len(tagged_essays2)

226

In [25]:
for essay in tagged_essays2[0:20]:
    print("*" * 80)
    print(essay.name)
    print("*" * 80)
    sents = []
    for sent in essay.sentences:
        tokens = []
        for i, (wd, tags) in enumerate(sent):
            if i == 0:
                wd = wd[0].upper() + wd[1:]
            tokens.append(wd.strip())
        str_sent = " ".join(tokens).strip()
        sents.append(str_sent)
    str_essay = "\n>>> ".join(sents)
    print(str_essay)

********************************************************************************
EBA1415_AEKD_4_CB_ES-05574.ann
********************************************************************************
Well based on what i read the corals are loosing their colors , coral bleaching are a serious problem with a serious impact on the worlds coral reefs .
>>> This is a serious problem because the coral bleaching is most noticeable in the pacific ocean , the ocean covers about 0 0 of the entire globe , some corals are sensitive to how salty the water us .
>>> A massive coral bleaching event in 0000 is one of the worst ever obscure , the event resulted in the death of 00 % of the worlds coral reefs , thus very serious it killed a lot of coral reefs .
********************************************************************************
EBA1415_AEKD_5_CB-06232.ann
********************************************************************************
Marinos biologists personas different types of corals living in 

In [26]:
output_folder = root_folder + "CoReference/" + PARTITION
output_folder

'/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/Test'

In [27]:
for essay in tagged_essays2:

    sents = []
    for sent in essay.sentences:
        tokens = []
        for i, (wd, tags) in enumerate(sent):
            if i == 0:
                wd = wd[0].upper() + wd[1:]
            tokens.append(wd.strip())
        str_sent = " ".join(tokens).strip()
        sents.append(str_sent)
    str_essay = "\n".join(sents)
    fname = output_folder + "/" + essay.name.replace(".ann", ".txt")
    #print(fname)
    #continue
    with open(fname, "w+") as f:
        f.write(str_essay)

In [12]:
# e = [e for e in tagged_essays2 if e.name.startswith("EBA1415_AEKD_5_CB_ES-05579")][0]
# e

In [13]:
# # Validate it doesn't replace biology with INFREQUENT
# for sent in e.sentences[0:1]:
#     print("*" * 80)
#     wds = list(zip(*sent))[0]
#     print(" ".join(wds))

## NOTES
* Don't replace INFREQUENT's - may break co-reference resolution, and also RNN tagging
* Need to lower case words after processing